In [12]:
import numpy as np
import pandas as pd

# CONSTANTS
n = 0          
g = 20        # Degrees of freedom
c1 = np.sqrt(4*(np.pi**3)/45)
c2 = 2*(np.pi**2)/45
DecayConstant = 1e-4
M_Pl = 1.2206e22  # Planck mass in MeV

In [14]:
# COUPLING CONSTANTS
k_SC, k_SV = 1e-6,1e-7 #Talk to Evan and change these numbers
k_Point = 1e-3


# PARTICLE VELOCITIES
V_Pion = 54 #m/s 
V_Kaon = 54
V_Baryon = 54

# Particle masses in MeV
#m = 0.511e-3      # Electron mass in GeV
M_Pl = 1.2206e19  # Planck mass in GeV
M_Proton, M_Neutron, M_Lambda = 0.9382720813, 0.93957, 1.115683
M_ChargedPion, M_NeutralPion = 0.13957039, 0.1349768
M_ChargedKaon, M_NeutralKaon = 0.493677, 0.497611

#Testing pion masses to measure how mass effects these calcs
M_NeutralPion = 1e-1
M_NeutralKaon = 4e-1

Test_baryon = 1e-1 #MeV
Test_meson = 1e0
k_test = 1e-2

In [16]:
# Load the data from CSV files
Gstar_df = pd.read_csv("Gstar.csv")
GstarS_df = pd.read_csv("GstarS.csv")

Gstar = dict(zip(Gstar_df.iloc[:, 0], Gstar_df.iloc[:, 1]))
GstarS = dict(zip(GstarS_df.iloc[:, 0], GstarS_df.iloc[:, 1]))

def g_star(Temp):
    # Find closest temperature value in Gstar
    closest_temp = min(Gstar.keys(), key=lambda t: abs(t - Temp))
    return Gstar[closest_temp]

def g_starS(Temp):
    # Find closest temperature value in GstarS
    closest_temp = min(GstarS.keys(), key=lambda t: abs(t - Temp))
    return GstarS[closest_temp]

In [18]:
def YInitial(particle):
    mass = particle_mass.get(particle, 1)
    return 8e-12 * (100/mass)

def n_eq(x, particle):
    mass = particle_mass.get(particle, 1)
    T = mass/x
    return (g/(2*(np.pi**2)))*(mass**2)*(T)*kn(2, x)

def Y_EQ(x, particle):
    mass = particle_mass.get(particle, 1)
    return n_eq(x, particle) / s(x, particle)

# Define H(m) function (Hubble parameter)
def H_m(x, particle):
    mass = particle_mass.get(particle, 1)
    return (c1 * np.sqrt(g_star(mass / x)) * (mass / x)**2) / M_Pl

# Define s(x, m) function (entropy density)
def s(x, particle):
    mass = particle_mass.get(particle, 1)
    g_starS_vec = np.vectorize(g_starS) 
    return c2 * g_starS_vec(mass / x) * (mass / x)**3
    #return c2 * g_starS(mass / x) * (mass / x)**3

def time(x, particle):
    m = particle_mass.get(particle, 1)
    T = m/x
    g_s = np.vectorize(g_star)(T)
    return 0.301*(g_s**(-1/2))*M_Pl/(T**2)

In [13]:
# Cross-section formula for a single Lambda
def sigma(s, particle):
    #cross_section = particle_cross_sections.get(particle, 1)
    mass = particle_mass.get("NeutralPion",1)
    k = particle_coupling_constants.get("NeutralPion",1)
    return ((k**2)*(mass**2))/(s)
    #return sum(Lambda / (2 * s) for Lambda in lambdas) #Make an array that has lambdas_pion, lambdas_proton, lambdas_neutron, etc. for all of their interactions
"""    
def thermally_averaged_cross_section(x, particle):
    mass = particle_mass.get(particle, 1)
    T = mass/x
    def integrand(s):
        return sigma(s, particle) * np.sqrt(s) * kn(1, np.sqrt(s) / T)
    
    integral_result, error = quad(integrand, 4 * mass**2, np.inf)

    return (1 / (8 * mass**4 * T * kn(2, x))) * integral_result
"""
# Function to compute prefactor k
def fk(particle):
    mass = particle_mass.get(particle, 1)
    cross_section = particle_cross_sections.get(particle, 1)
    return np.sqrt(np.pi * M_Pl**2 / 45) * mass * cross_section

# Function to compute variables in the Boltzmann equation
def Variables(x, particle):
    mass = particle_mass.get(particle, 1)
    T = mass / x
    return k * g_star(T) / x**2  # g_star(T) is assumed to be defined elsewhere

# Boltzmann equation in log space
def dYdx_log(x, logY, particle):
    Y = np.exp(logY)
    ratio = particle_ratios.get(particle, 1)
    x_ratio = max(0.01, x * ratio)
    #x_ratio = x * ratio
    return -((Y**2 - Y_EQ(x_ratio, particle)**2) * Variables(x, particle)) / Y # d(logY)/dx


# Equilibrium abundance function (assumed to be pre-defined)
def Y_EQ(x, particle):
    
    #ratio = particle_ratios.get(particle, 1)
    #x_ratio = max(0.1, x * ratio)

    return 0.145 * (x**(3/2)*0.5) * np.exp(-x*0.5)


    
"""
def fk(x, particle):
    mass = particle_mass.get(particle,1)
    T = mass/x
    cross_section = thermally_averaged_cross_section(x, particle)
    return np.sqrt(np.pi*(M_Pl**2)/45)*mass*cross_section

def Variables(x, particle):
    mass = particle_mass.get(particle,1)
    T=mass/x
    k = fk(x,particle)
    return k*np.sqrt(g_star(T))/x**2

def dYdx(Y,x,particle): # Eqn 7.10 of https://arxiv.org/pdf/1009.3690
    ratio = particle_ratios.get(particle, 1)  
    x_ratio = x*ratio
    
    return -(Y**2-Y_EQ(x_ratio, particle)**2)*Variables(x_ratio, particle)
"""
def get_variable_name(var, global_vars):
    names = [name for name, value in global_vars.items() if value is var]
    return names[0] if names else "unknown"

In [ ]:
def thermally_averaged_cross_section(x,particle):
    c_chi = particle_coupling_constants.get("NeutralPion")
    m_chi = particle_mass.get(particle,1)
    f_a = particle_mass.get("NeutralPion")
    T = m_chi/x
    return ((c_chi**4) * (m_chi) * T) / (64*np.pi*(f_a**4))
    
# Function to compute prefactor k
def fkdynamic(x,particle):
    mass = particle_mass.get(particle, 1)
    cross_section = thermally_averaged_cross_section(x,particle)
    return np.sqrt(np.pi * M_Pl**2 / 45) * mass * cross_section

# Function to compute variables in the Boltzmann equation
def Variablesdynamic(x, particle):
    mass = particle_mass.get(particle, 1)
    T = mass / x
    k = fkdynamic(x,particle)
    return k * g_star(T) / x**2  # g_star(T) is assumed to be defined elsewhere


def dYdx_logdynamic(x, logY, particle):
    Y = np.exp(logY)
    ratio = particle_ratios.get(particle, 1)
    x_ratio = max(0.01, x * ratio)
    return -((Y**2 - Y_EQ(x_ratio, particle)**2) * Variablesdynamic(x_ratio, particle)) / Y # d(logY)/dx

In [ ]:
def dYdx_logdynamicWithDecays(x, logY, particle):
    Y = np.exp(logY)
    ratio = particle_ratios.get(particle, 1)
    x_ratio = max(0.01, x * ratio)
    
    dYdx = -((Y**2 - Y_EQ(x_ratio, particle)**2) * Variablesdynamic(x_ratio, particle)) / Y 

    if particle in decay_products:
        decay_rate = AvgDecayRate(x)
        for products in decay_products[particle]:
            for product in products:
                dYdx += 0.5 * decay_rate * Y  # 50% branching fraction
            
    return dYdx

In [19]:
def AvgDecayRate(x):
    ratio = particle_ratios.get(particle, 1)
    x_ratio = x*ratio
    return np.exp(-DecayConstant*x_ratio) #Ae^-lambda*t but using x instead

def dYdxRadio(Y,x,particle):
    ratio = particle_ratios.get(particle, 1)  
    mass = particle_mass.get(particle,1)

    x_ratio = x*ratio
    T = mass/x
    
    return -(Y**2-Y_EQ(x_ratio, particle)**2)*Variables(x_ratio, particle) - (1.661*x_ratio)/((mass**2)*np.sqrt(g_star(T)))*AvgDecayRate(x_ratio)*(Y-Y_EQ(x_ratio,particle)*(1)**2)

def CoupleddYdx(x, Y, Y_prod, AvgDecayRate, Lambda):
    Y_eq = Y_EQ(x)  
    lambda_val = lambda_x(x, m, g_starS, g_star, Lambda)
    AvgDecayRate_val = AvgDecayRate(x)
    
    if x > 100:  # Arbitrary cutoff for equilibrium; modify as needed
        return np.array([0])
    #NOTE THAT YOU NEED TO DEFINE A DECAY RATE, Y_PROD AND Y_PROD_EQ
    result = -lambda_val * x**(-n-2) * (Y**2 - Y_eq**2) - ((1.661*x)/((m**2)*(g_star**1/2))*AvgDecayRate_val*(Y-Y_eq*(Y_prod/Y_prod_eq)**2)) 
    return np.array([result])

def dYproddx(x, Y, Y_prod, AvgDecayRate, Lambda):
    AvgDecayRate_val = AvgDecayRate(x)
    
    if x > 100:  # Arbitrary cutoff for equilibrium; modify as needed
        return np.array([0])
        
    result = - ((1.661*x)/((m**2)*(g_star**1/2))*AvgDecayRate_val*(Y-Y_eq*(Y_prod/Y_prod_eq)**2))
    return np.array([result])

# To do list
##### Want sum(m_P * Y) = 4e-10 GeV so all the particle masses multiplied by the relic abundance should be equal to 4e-10 GeV

In [22]:
#def sigma(s, Lambda):
#    return Lambda / (2 * s)

"""
# Differential equation for relic abundance
def dYdx(x, Y, Lambda):
    Y_eq = Y_EQ(x)  
    lambda_val = lambda_x(x, m, g_starS, g_star, Lambda)
    
    if x > 100:  # Arbitrary cutoff for equilibrium; modify as needed
        return np.array([0])
    
    result = -lambda_val * x**(-n-2) * (Y**2 - Y_eq**2)
    return np.array([result])
"""

'\n# Differential equation for relic abundance\ndef dYdx(x, Y, Lambda):\n    Y_eq = Y_EQ(x)  \n    lambda_val = lambda_x(x, m, g_starS, g_star, Lambda)\n    \n    if x > 100:  # Arbitrary cutoff for equilibrium; modify as needed\n        return np.array([0])\n    \n    result = -lambda_val * x**(-n-2) * (Y**2 - Y_eq**2)\n    return np.array([result])\n'

In [55]:
"""
def Y_EQ_GT3(x, particle): #x >> 3
    mass = particle_mass.get(particle, 1) # Get particle masses, default to 1 if not found
    return 0.145 * (g / g_starS(mass / x)) * x**(3/2) * np.exp(-x) #eqn 5.25

def Y_EQ_LT3(x, particle): #x << 3
    mass = particle_mass.get(particle, 1) # Get particle masses, default to 1 if not found
    return 0.278 * (g / g_starS(mass / x)) #eqn 5.25
"""

"""   
# Lambda_x function for freeze-out dynamics
def lambda_x(x, g_s, g_star, particle):
    H_m_value = H_m(x, particle)  
    mass = particle_mass.get(particle, 1)
    T = mass / x
    thermally_avg_sigma = thermally_averaged_cross_section(T, particle)
    s_value = s(x, particle)
    return (x * thermally_avg_sigma * s_value) / H_m_value # Equation 1.57 in my thesis notes.

# Differential equation for relic abundance
def dYdx(x, Y, particle):
    ratio = particle_ratios.get(particle, 1)  

    x_ratio = x*ratio

    Y_eq = Y_EQ(x_ratio, particle)
    lambda_val = lambda_x(x_ratio, g_starS, g_star, particle)

#    if x > 100:  # Arbitrary cutoff for equilibrium
#        return np.array([0])
    
    result = -lambda_val * (Y**2 - Y_eq**2)
    return np.asarray([result])
"""